# Kernel Ridge Regression

In [21]:
%matplotlib inline
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import pickle
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV

# environment settings
data_path = 'Data/'

# Deserialize previously saved data from "preprocessing"
with open(data_path+'train_pp.obj', 'rb') as train_pp, \
open(data_path+'test_pp.obj','rb') as test_pp:
    train_df = pickle.load(train_pp)
    test_df = pickle.load(test_pp)
train_df["SalePrice"] = np.log1p(train_df["SalePrice"])

In [22]:
X = train_df.loc[:,'Id':'SaleCondition_Partial']
y = train_df['SalePrice']
print("Shape of training set {}.\nShape of test set {}".format(X.shape,y.shape))

Shape of training set (1460, 303).
Shape of test set (1460,)


In [23]:
KRR = KernelRidge(alpha=10,gamma=1, kernel='polynomial', degree=1)
kf = KFold(5, shuffle=True, random_state=42).get_n_splits(X)
rmse_cv= np.sqrt(-cross_val_score(KRR, X, y, scoring="neg_mean_squared_error", cv = kf))

print("The 10-fold crossvalidation RMSE is {:.5f} +/- {:.3f} ".format(rmse_cv.mean(),rmse_cv.std()))

The 10-fold crossvalidation RMSE is 0.13981 +/- 0.026 


In [18]:
#Prediction on real test set using KRR
KRR = KernelRidge(alpha=10.0,gamma=1, kernel='linear', degree=1)
KRR.fit(X,y)
pred_results =KRR.predict(test_df)
result_df = pd.DataFrame(data={'Id': test_df["Id"].values,
                               'SalePrice': pred_results})
#Create output csv file
result_df.to_csv(data_path+"outputs/kernel_ridge_a_1", index=False)